<a href="https://colab.research.google.com/github/FedorSavchenko/04.HW_KAGGLE_RSP_/blob/main/04_HW_KaggleAgents_RPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install -q -U kaggle_environments

In [31]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

#Bot01-incremental
примитивный агент который циклично выдает значения 

In [37]:
%%writefile incremental_agent.py
import random
incr_state = 0
def agent(observation, configuration):
  global incr_state
  if observation.step == 0:
    incr_state = random.randrange(0, configuration.signs) 
  incr_state += 1
  return incr_state % configuration.signs

Overwriting incremental_agent.py


#Bot02-Decremental
Bot01 не получил ни одного поражения, хотя побед не много, но занимает увернное 4е место в турнироной таблице. Проверим реверсивный порядок цикла.

In [33]:
%%writefile decremental_agent.py
import random
incr_state = 0
def agent(observation, configuration):
  global incr_state
  if observation.step == 0:
    incr_state = random.randrange(0, configuration.signs) 
  incr_state += 2
  return incr_state % configuration.signs

Overwriting decremental_agent.py


#Bot03-incremental++
турнирная таблица показала что

А) У Bot01 нет ни одного пораженя но побеждает он только одного агента - copy_opponent

B) У Bot02 побед больше он побеждает агента reactionary, который в топе таблицы, но проигрывает агненту copy_opponent,  так же с агентом counter_reactionary в споре,где влияет случайность первого хода

Попытаемся улучшить агента Bot01, при этом оставить его без поражения, добавив побед над stone, rock, scissors



In [40]:
%%writefile incremental_plusplus.py
import random

TESTING_STEPS = 10
incr_state = 0
opponent_histogram = None

def agent(observation, configuration):
  
  global incr_state
  global opponent_histogram
  if(opponent_histogram is None):
    opponent_histogram = [0]*configuration.signs
  
  if observation.step == 0:
    #first turn
    incr_state = random.randrange(0, configuration.signs) 
    result = incr_state
  elif observation.step < TESTING_STEPS or max(opponent_histogram) != observation.step-1:
    #researching or first TESTING_STEPS were't same turn value and use incremental tactic
    incr_state += 1
    result = incr_state % configuration.signs
  else:
    # scissors, rock paper bots
    opponent_turns = opponent_histogram.index(max(opponent_histogram))
    result = (opponent_turns + 1) % configuration.signs
  if observation.step > 0:
    opponent_action = observation.lastOpponentAction
    opponent_histogram[opponent_action] += 1

  return result

Writing incremental_plusplus.py



#Bot04-counter_statistical
Bot03 - с легкостью победив 3х ботов с константой выдачей, попадает на первое место по количеству очков и на второе место по количеству побед.:D Нужно создать несколько тактик против других ботов.
Для этого сделаем бота-противника для statistical

In [35]:
#%%writefile counter_statistic.py
import random

TESTING_STEPS = 3
awaiting_statistic_turn = 0
my_histogram = None

def agent(observation, configuration):
  global awaiting_statistic_turn
  global my_histogram

  if(my_histogram is None):
    my_histogram = [0]*configuration.signs

  if observation.step == 0:
    result = random.randrange(0, configuration.signs) 
  else:
    result = (awaiting_statistic_turn + 1) % configuration.signs

  my_histogram[result] += 1
  awaiting_statistic_turn = (my_histogram.index(max(my_histogram)) + 1) % configuration.signs

  return result

Overwriting counter_statistic.py


In [39]:
evaluate(
      "rps", 
      ["counter_statistic.py", "statistical"], 
      configuration={"episodeSteps": 100},
      num_episodes = 10
    )


[[0, 0],
 [66.0, -66.0],
 [66.0, -66.0],
 [0, 0],
 [66.0, -66.0],
 [0, 0],
 [0, 0],
 [66.0, -66.0],
 [0, 0],
 [66.0, -66.0]]


#Bot05-counter_statistic+
Bot04 - С вероятностью 66% побеждает statistical, как побочный эффект побеждает reactional and counter_reactional, но при этом проигрывает боту copy_opponent. Улучшим его и поставим защиту от copy opponent

In [49]:
%%writefile counter_statistic_plus.py
import random

TESTING_STEPS = 10

opponent_tactics = {
    "statistic":0,
    "copy":0,
    "defence":0
}

awaiting_statistic_turn = 0
awaiting_defence_turn = 0
awaiting_copy_turn = 0

incr_state = 0
my_histogram = None

def agent(observation, configuration):
  global incr_state
  global awaiting_statistic_turn
  global awaiting_defence_turn
  global awaiting_copy_turn
  global my_histogram
  global opponent_tactics
  
  result = 0 
  if(my_histogram is None):
    my_histogram = [0]*configuration.signs
  
  #first turn
  if observation.step == 0:
    incr_state = random.randrange(0, configuration.signs - 1) 
    result = incr_state
  
  #collecting statistic
  elif observation.step < TESTING_STEPS:
    incr_state += 1
    result = incr_state % configuration.signs
  else:
    predicted_tactic = max(opponent_tactics.items(), key = lambda e:e[1])[0]
    
    if predicted_tactic == "statistic":
      result = (awaiting_statistic_turn + 1) % configuration.signs
    
    elif predicted_tactic == "copy":
      result = awaiting_statistic_turn % configuration.signs
    
    elif predicted_tactic == "defence":
      result = (awaiting_statistic_turn + 2) % configuration.signs

  #calc opponent tactic stat
  if(observation.step > 0):
    opponent_action = observation.lastOpponentAction
    if(opponent_action == awaiting_statistic_turn):
      opponent_tactics["statistic"] += 1
  
    if(opponent_action == awaiting_defence_turn):
      opponent_tactics["defence"] += 1

    if(opponent_action == awaiting_copy_turn):
      opponent_tactics["copy"] += 1
  

  #calc my stat
  my_histogram[result] += 1
  my_hist_maxsign = my_histogram.index(max(my_histogram))
  awaiting_statistic_turn = (my_hist_maxsign + 1) % configuration.signs
  awaiting_defence_turn = (my_hist_maxsign + 2) % configuration.signs
  awaiting_copy_turn = my_hist_maxsign  % configuration.signs

  return result

Writing counter_statistic_plus.py


In [51]:
evaluate(
      "rps", 
      #["incremental_Alpha.py", "statistical"], 
      ["counter_statistic_plus.py", "copy_opponent"], 
      configuration={"episodeSteps": 100},
      num_episodes = 10
    )


[[47.0, -47.0],
 [47.0, -47.0],
 [51.0, -51.0],
 [53.0, -53.0],
 [48.0, -48.0],
 [53.0, -53.0],
 [53.0, -53.0],
 [53.0, -53.0],
 [48.0, -48.0],
 [49.0, -49.0]]

#Bot06-MultyTacticAnalyse
Bot05 - Справляется с ботами copy_opponent и statistical. Добавим логику Bot03,чтоб стравлялся с константными.

In [64]:
%%writefile multy_tactic_analyse.py
import random

TESTING_STEPS = 10

opponent_tactics = {
    "statistic":0,
    "copy":0,
    "defence":0,
    "repeat":0,
}

awaiting_statistic_turn = 0
awaiting_defence_turn = 0
awaiting_copy_turn = 0

incr_state = 0
my_histogram = None

opponent_old_action = 0

def agent(observation, configuration):
  global incr_state
  global awaiting_statistic_turn
  global awaiting_defence_turn
  global awaiting_copy_turn
  global my_histogram
  global opponent_tactics
  global opponent_old_action
  
  result = 0 
  if(my_histogram is None):
    my_histogram = [0]*configuration.signs
  
  #first turn
  if observation.step == 0:
    incr_state = random.randrange(0, configuration.signs - 1) 
    result = incr_state
  
  #collecting statistic
  elif observation.step < TESTING_STEPS:
    incr_state += 1
    result = incr_state % configuration.signs
  else:
    predicted_tactic = max(opponent_tactics.items(), key = lambda e:e[1])[0]
    
    if predicted_tactic == "statistic":
      result = (awaiting_statistic_turn + 1) % configuration.signs
    
    elif predicted_tactic == "copy":
      result = awaiting_statistic_turn % configuration.signs
    
    elif predicted_tactic == "defence":
      result = (awaiting_statistic_turn + 2) % configuration.signs

    elif predicted_tactic == "repeat":
      result = (opponent_old_action + 1) % configuration.signs


  #calc opponent tactic stat
  if(observation.step > 0):
    opponent_action = observation.lastOpponentAction
    if(opponent_action == opponent_old_action):
      opponent_tactics["repeat"] += 1

    if(opponent_action == awaiting_statistic_turn):
      opponent_tactics["statistic"] += 1
  
    if(opponent_action == awaiting_defence_turn):
      opponent_tactics["defence"] += 1

    if(opponent_action == awaiting_copy_turn):
      opponent_tactics["copy"] += 1

    opponent_old_action = opponent_action
  

  #calc my stat
  my_histogram[result] += 1
  my_hist_maxsign = my_histogram.index(max(my_histogram))
  awaiting_statistic_turn = (my_hist_maxsign + 1) % configuration.signs
  awaiting_defence_turn = (my_hist_maxsign + 2) % configuration.signs
  awaiting_copy_turn = my_hist_maxsign  % configuration.signs

  return result

Writing multy_tactic_analyse.py


#Bot07-MultyTacticAnalyse_2
Bot06 - Бесспорный лидер, по очкам, победам, при этом нет ни одного поражения. Добавим логику чтоб справлялся с Incremenal и Decremental 

In [62]:
%%writefile multy_tactic_analyse_2.py
import random

TESTING_STEPS = 10

opponent_tactics = {
    "statistic":0,
    "copy":0,
    "defence":0,
    "repeat":0,
    "increment":0,
    "decrement":0,
}

awaiting_statistic_turn = 0
awaiting_defence_turn = 0
awaiting_copy_turn = 0

incr_state = 0
my_histogram = None

opponent_old_action = 0

def agent(observation, configuration):
  global incr_state
  global awaiting_statistic_turn
  global awaiting_defence_turn
  global awaiting_copy_turn
  global my_histogram
  global opponent_tactics
  global opponent_old_action
  
  result = 0 
  if(my_histogram is None):
    my_histogram = [0] * configuration.signs
  
  #first turn
  if observation.step == 0:
    incr_state = random.randrange(0, configuration.signs - 1) 
    result = incr_state
  
  #till collecting statistic
  elif observation.step < TESTING_STEPS:
    incr_state += 1
    result = incr_state % configuration.signs
  else:
    predicted_tactic = max(opponent_tactics.items(), key = lambda e:e[1])[0]
    
    if predicted_tactic == "statistic":
      result = (awaiting_statistic_turn + 1) % configuration.signs
    
    elif predicted_tactic == "copy":
      result = awaiting_statistic_turn % configuration.signs
    
    elif predicted_tactic == "defence":
      result = (awaiting_statistic_turn + 2) % configuration.signs

    elif predicted_tactic == "repeat":
      result = (opponent_old_action + 1) % configuration.signs

    elif predicted_tactic == "increment":
      result = opponent_old_action % configuration.signs
    
    elif predicted_tactic == "decrement":
      result = (opponent_old_action + 2) % configuration.signs

  #calc opponent tactic stat
  if observation.step > 0 :
    opponent_action = observation.lastOpponentAction
    if opponent_action == opponent_old_action :
      opponent_tactics["repeat"] += 1
    
    if opponent_action == (opponent_old_action + 1) % configuration.signs:
      opponent_tactics["increment"] += 1

    if opponent_action == (opponent_old_action + 2) % configuration.signs:
      opponent_tactics["decrement"] += 1

    if opponent_action == awaiting_statistic_turn:
      opponent_tactics["statistic"] += 1
  
    if(opponent_action == awaiting_defence_turn):
      opponent_tactics["defence"] += 1

    if(opponent_action == awaiting_copy_turn):
      opponent_tactics["copy"] += 1

    opponent_old_action = opponent_action
  

  #calc my stat
  my_histogram[result] += 1
  my_hist_maxsign = my_histogram.index(max(my_histogram))
  awaiting_statistic_turn = (my_hist_maxsign + 1) % configuration.signs
  awaiting_defence_turn = (my_hist_maxsign + 2) % configuration.signs
  awaiting_copy_turn = my_hist_maxsign  % configuration.signs

  return result

Writing multy_tactic_analyse_2.py


In [63]:
evaluate(
      "rps", 
      #["incremental_Alpha.py", "statistical"], 
      ["multy_tactic_analyse_2.py", "decremental_agent.py"], 
      configuration={"episodeSteps": 100},
      num_episodes = 10
    )

[[88.0, -88.0],
 [88.0, -88.0],
 [90.0, -90.0],
 [89.0, -89.0],
 [89.0, -89.0],
 [89.0, -89.0],
 [88.0, -88.0],
 [89.0, -89.0],
 [88.0, -88.0],
 [89.0, -89.0]]


#Bot08-testBot_Random
Bot07 - Превзошел все ожидания одержав 29 побед из 33, где 2 ничьи и 2 поражения своим младшим братьям. Но теперь нужно проверить настолько ли он хорош на самом деле. Создадим ему разных оппоненов. Куда же нам без рандом бота)

In [61]:
%%writefile random_agent.py
import random
def agent(observation, configuration):
  return random.randrange(0, configuration.signs) 

Writing random_agent.py


#Bot09-testBot_WrongOrder_1
к самому сильному боту Bot07, создаем различные тесты.

0,1,2,1,0,1,2,1,0...

In [60]:
%%writefile wrong_order_1.py
increment = 0
order = [0,1,2,1]
def agent(observation, configuration):
  global increment
  increment += 1
  return order[increment % len(order)]

Writing wrong_order_1.py


#Bot10-testBot_WrongOrder_2
к самому сильному боту Bot07, создаем различные тесты.

0,0,0,0,1,1,1,2,2

In [59]:
%%writefile wrong_order_2.py
increment = 0
order = [0,0,0,0,1,1,1,2,2]
def agent(observation, configuration):
  global increment
  increment += 1
  return order[increment % len(order)]

Writing wrong_order_2.py


#Bot11-testBot_WrongOrder_3
к самому сильному боту Bot07, создаем различные тесты.

0,2,1,0,1,0,2,1

In [58]:
%%writefile wrong_order_3.py
increment = 0
order = [0,2,1,0,1,0,2,1]
def agent(observation, configuration):
  global increment
  increment += 1
  return order[increment % len(order)]

Writing wrong_order_3.py


#Bot12-AntiBoss 
попробуем сделать противника для Bot07, добавив к результату в конце 1, чтоб вернуть выигрышную коибинацию


In [66]:
%%writefile anti_boss.py
import random

TESTING_STEPS = 10

opponent_tactics = {
    "statistic":0,
    "copy":0,
    "defence":0,
    "repeat":0,
    "increment":0,
    "decrement":0,
}

awaiting_statistic_turn = 0
awaiting_defence_turn = 0
awaiting_copy_turn = 0

incr_state = 0
my_histogram = None

opponent_old_action = 0

def agent(observation, configuration):
  global incr_state
  global awaiting_statistic_turn
  global awaiting_defence_turn
  global awaiting_copy_turn
  global my_histogram
  global opponent_tactics
  global opponent_old_action
  
  result = 0 
  if(my_histogram is None):
    my_histogram = [0] * configuration.signs
  
  #first turn
  if observation.step == 0:
    incr_state = 0#random.randrange(0, configuration.signs - 1) 
    result = incr_state
  
  #till collecting statistic
  elif observation.step < TESTING_STEPS:
    incr_state += 1
    result = incr_state % configuration.signs
  else:
    predicted_tactic = max(opponent_tactics.items(), key = lambda e:e[1])[0]
    
    if predicted_tactic == "statistic":
      result = (awaiting_statistic_turn + 1) % configuration.signs
    
    elif predicted_tactic == "copy":
      result = awaiting_statistic_turn % configuration.signs
    
    elif predicted_tactic == "defence":
      result = (awaiting_statistic_turn + 2) % configuration.signs

    elif predicted_tactic == "repeat":
      result = (opponent_old_action + 1) % configuration.signs

    elif predicted_tactic == "increment":
      result = opponent_old_action % configuration.signs
    
    elif predicted_tactic == "decrement":
      result = (opponent_old_action + 2) % configuration.signs

  #calc opponent tactic stat
  if observation.step > 0 :
    opponent_action = observation.lastOpponentAction
    if opponent_action == opponent_old_action :
      opponent_tactics["repeat"] += 1
    
    if opponent_action == (opponent_old_action + 1) % configuration.signs:
      opponent_tactics["increment"] += 1

    if opponent_action == (opponent_old_action + 2) % configuration.signs:
      opponent_tactics["decrement"] += 1

    if opponent_action == awaiting_statistic_turn:
      opponent_tactics["statistic"] += 1
  
    if(opponent_action == awaiting_defence_turn):
      opponent_tactics["defence"] += 1

    if(opponent_action == awaiting_copy_turn):
      opponent_tactics["copy"] += 1

    opponent_old_action = opponent_action
  

  #calc my stat
  result = (result + 1) % configuration.signs
  my_histogram[result] += 1
  my_hist_maxsign = my_histogram.index(max(my_histogram))
  awaiting_statistic_turn = (my_hist_maxsign + 1) % configuration.signs
  awaiting_defence_turn = (my_hist_maxsign + 2) % configuration.signs
  awaiting_copy_turn = my_hist_maxsign  % configuration.signs

  return result

Overwriting anti_boss.py


In [67]:
evaluate(
      "rps", 
      ["anti_boss.py","multy_tactic_analyse_2.py"], 
      configuration={"episodeSteps": 100},
      num_episodes = 10
    )

[[0, 0],
 [54.0, -54.0],
 [0, 0],
 [54.0, -54.0],
 [54.0, -54.0],
 [0, 0],
 [0, 0],
 [0, 0],
 [54.0, -54.0],
 [0, 0]]

# Регистрация Агентов к турниру

In [54]:
kaggle_agents = [
    "rock", 
    "paper", 
    "scissors",
    "copy_opponent",
    "reactionary",
    "counter_reactionary",
    "statistical"
]

my_agents = [
    "incremental_agent.py",
    "decremental_agent.py",
    "incremental_plusplus.py",
    "counter_statistic.py",
    "counter_statistic_plus.py",
    "multy_tactic_analyse.py",
    "multy_tactic_analyse_2.py",
    "random_agent.py",
    "wrong_order_1.py",
    "wrong_order_2.py",
    "wrong_order_3.py",
    "anti_boss.py"
]

agent_names =  my_agents + kaggle_agents

#better to use clsss here but it is out of study scope now
def create_agent_result(name):
  obj = lambda:None
  obj.name = name
  def reset():
    obj.score = 0
    obj.wins = 0
    obj.plays = 0
    obj.draws = 0
    obj.stronger_oppenents = set()  
  obj.reset = reset
  reset()

  obj.to_string = lambda:f"\
    {(obj.name + ' '*20)[:25]} \
    \tscore:{obj.score}, \
    \tw|d|l: {obj.wins}, {obj.draws}, {obj.plays - obj.wins - obj.draws}\
    \nlosed to:{obj.stronger_oppenents}"
  return obj

reward_dict = {}
reward_list = [] #for sorting

for i in agent_names:
  new_agent = create_agent_result(i)
  reward_dict[i] = new_agent;
  reward_list.append(new_agent)
  print(new_agent.to_string(),"\n")
print("Agents Ready. Here we go!")

    incremental_agent.py          	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    decremental_agent.py          	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    incremental_plusplus.py       	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    counter_statistic.py          	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    counter_statistic_plus.py     	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    multy_tactic_analyse.py       	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    multy_tactic_analyse_2.py     	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    random_agent.py               	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    wrong_order_1.py              	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    wrong_order_2.py              	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    wrong_order_3.py              	score:0,     	w|d|l: 0, 0, 0    
losed to:set() 

    anti_boss.py                  	score:0,     	w|d|l: 0, 0, 0  

#Турнир

In [65]:
ATTEMTS_COUNT = 100
BATTLES = 3

#reset all result values
for i in reward_dict.values(): i.reset()

#battles
for i in range(len(agent_names)):
  for j in range(i+1, len(agent_names)):
    agent_A = reward_dict[agent_names[i]]
    agent_B = reward_dict[agent_names[j]]

    battles = evaluate(
      "rps", 
      [agent_A.name, agent_B.name], 
      configuration={"episodeSteps": ATTEMTS_COUNT},
      num_episodes = BATTLES
    )

    for battle in battles:
      agent_A.score += battle[0]
      agent_B.score += battle[1]

      if battle[0] > 0 :
        agent_A.wins += 1
        agent_B.stronger_oppenents.add(agent_A.name)
      elif battle[1] > 0 :
        agent_B.wins += 1
        agent_A.stronger_oppenents.add(agent_B.name)
      else:
        agent_A.draws += 1
        agent_B.draws += 1

      agent_A.plays += 1
      agent_B.plays += 1

reward_list.sort(key = lambda e: (e.wins, e.score), reverse = True)
for i,item in enumerate(reward_list): 
  print(i+1,item.to_string(),"", sep= ".")

1.    multy_tactic_analyse_2.py     	score:2359.0,     	w|d|l: 40, 9, 5    
losed to:{'multy_tactic_analyse.py', 'counter_statistic_plus.py', 'anti_boss.py'}.

2.    multy_tactic_analyse.py       	score:1309.0,     	w|d|l: 21, 33, 0    
losed to:set().

3.    statistical                   	score:602.0,     	w|d|l: 21, 18, 15    
losed to:{'reactionary', 'multy_tactic_analyse_2.py', 'multy_tactic_analyse.py', 'counter_statistic_plus.py', 'counter_statistic.py'}.

4.    reactionary                   	score:149.0,     	w|d|l: 20, 17, 17    
losed to:{'wrong_order_3.py', 'multy_tactic_analyse_2.py', 'counter_reactionary', 'wrong_order_1.py', 'copy_opponent', 'decremental_agent.py', 'counter_statistic.py'}.

5.    incremental_plusplus.py       	score:1120.0,     	w|d|l: 18, 31, 5    
losed to:{'incremental_agent.py', 'multy_tactic_analyse_2.py'}.

6.    counter_reactionary           	score:-209.0,     	w|d|l: 17, 20, 17    
losed to:{'wrong_order_3.py', 'multy_tactic_analyse_2.py', 'wrong_o